In [23]:
import numpy as np
import tracemalloc

# Todo List

- Create compact trie, to save space
    - CURRENT PROGRESS
    - Small change: to save space, maybe don't create a Node at every array index? Becuase we might overwrite it anyways
- Next array should only have intervals for two octaves up or down. If there is an interval stored that is larger than that, make it a linked list idea
    - So the main four octave region is like one big node, and then there are nodes added before or after as needed if larger intervals are required
- TODOs in code:
    - terminal value may need to be array
    - in search, return result before updating mostSearched
- BIG MILESTONE: combine with the html pianokeyboard
    - For flat vs. sharp, maybe use alt/option to toggle: https://stackoverflow.com/questions/13539493/how-to-detect-keyboard-modifier-ctrl-or-shift-through-javascript

# Basic Definitions and Classes

In [24]:
notes = {'C': 0, 'D': 2, 'E': 4, 'F': 5, 'G': 7, 'A': 9, 'B': 11}
accidentals = {'bb': -2, 'd': -2, 'b': -1, 'n': 0, '#': 1, '##': 2, 'x': 2}

maxRange = 96 - 9 # =87, the range from A0 to C8

Two representations of a single note:
1. note: string of the note name (`'E5'`)
2. value: the note value (`64`)

Three representations of a melody (string of notes):
1. melody: string of actual note names (`'E5 D#5 E5 D#5 E5'`)
2. values: list of note values (`[64, 63, 64, 63, 64]`)
3. intervals: list of the intervals between values (`[-1, 1, -1, 1]`)

In [25]:
def noteToValue(note):
    # get note, accidental, octave
    n = note[0]
    a = 'n' if len(note) == 2 else note[1:len(note)-1]
    o = int(note[-1])
    
    return notes[n] + accidentals[a] + o*12

In [26]:
def valuesToIntervals(vals):
    intervals = np.array([vals[i+1]-vals[i] for i in range(len(vals)-1)])
    return intervals

def melodyToValues(melody):
    # currently ignoring the slashes that indicate a rest/breath/phrase
    values = np.array([noteToValue(note) for note in melody.split() if note != '/'])
    return values

# shortcut function
def melodyToIntervals(melody):
    return valuesToIntervals(melodyToValues(melody))

In [27]:
class MusicalWork:
    def __init__(self, title, artist, date, genre, key, melody):
        self.title = title
        self.artist = artist
        self.date = date
        self.genre = genre
        
        self.melody = melody
        self.melodyValues = melodyToValues(melody)
        self.melodyIntervals = valuesToIntervals(self.melodyValues)
        self.key = key # key signature
        
        self.searchCount = 0
    
    def __str__(self):
        out =  f'[+{self.searchCount}] {self.title} | by {self.artist} ({self.date}, {self.genre})\n'
        out += f'Melody [{self.key}]: {self.melody}'
        return out

class ClassicalPiece(MusicalWork):
    def __init__(self, t, a, d, g, k, m, op=None, no=None, mv=None):
        MusicalWork.__init__(self, t, a, d, g, k, m)
        
        self.opus = op
        self.number = no
        self.movement = mv

        
class Song(MusicalWork):
    def __init__(self, t, a, d, g, k, m, aN=None):
        MusicalWork.__init__(self, t, a, d, g, k, m)
        
        self.albumName = aN

# Data Structure: Trie

In [28]:
# HELPER FUNCTIONS

# helper function to convert an interval to the correct index in nextArr
def ivlToIdx(ivl):
    return ivl + maxRange

# creates new nextArr
def createNewNextArr(prev=None, nodetype='compact'):
    if nodetype == 'compact':
        return np.array([CompactNode(i, prev) for i in range(-maxRange, maxRange+1)])
    else: # regular trie node
        return np.array([Node(i) for i in range(-maxRange, maxRange+1)])

# finds first nonmatching index
def firstNonmatching(base, test):
    '''
    Input: arrays base, test
    Output: first index at which the arrays differ, or
            -1 if all matching up to index min(len(base), len(test))
    '''
    # if either's length is 0 (i.e. None), return -1
    if base is None or test is None: return -1
    
    # go through each element, up to last element in smallest array
    for i in range(min(len(base), len(test))):
        if base[i] != test[i]: return i

    # if we got here, all were equal
    return -1

## Regular Trie

- Dense: array of next nodes (not sparse)
- One interval per node (not radix tree / compact trie)

In [29]:
class Node:
    def __init__(self, interval, nextArr=None):
        self.interval = interval
        self.nextArr = nextArr
        
        # for if there is an item that terminates at this node
        # TODO: may need to make this an array
        self.terminalValue = None
        
        # to keep track of most searched item
        # also used for matching with searches that are only first half of the melody, etc.
        self.mostSearched = None
    
    def __str__(self):
        hasNext = 'a' if self.nextArr is not None else 'no'
        tV_msg = self.terminalValue.title if self.terminalValue is not None else 'None'
        mS = self.mostSearched
        
        if self.mostSearched: mS_msg = f'{mS.title} with {mS.searchCount} searches'
        else: mS_msg = 'None'
        
        out = f'Interval {self.interval}, with {hasNext} nextArr\n'
        out += f'Terminal value of {tV_msg}\n'
        out += f'Most searched is {mS_msg}'
        return out

In [37]:
class Trie:
    def __init__(self):
        self.root = Node(None, createNewNextArr(nodetype='default'))
        self.numItems = 0
        
    def __str__(self):
        return f'Trie with {self.numItems} items'
    
    def __len__(self):
        return self.numItems
    
    
    def insert(self, mw):
        currNode = self.root
        
        # go through intervals (traverse trie)
        for ivl in mw.melodyIntervals:
            # populate currNodes's next array if necesary
            if currNode.nextArr is None: currNode.nextArr = createNewNextArr(nodetype='default')
            
            # update most searched for this node, if None
            if not currNode.mostSearched: currNode.mostSearched = mw
            
            # step down
            currNode = currNode.nextArr[ivlToIdx(ivl)]
        
        # currNode now holds terminal node for this musical work
        currNode.terminalValue = mw
        self.numItems += 1 # increment number of items
        
    def search(self, melody):
        # try to find the musical work
        # TODO: for speed reasons, find out how to return the result first,
        #       and only then update mostSearched?
        mw, exact = self._find(melody)
        
        
        # Determine if any Nodes' mostSearched needs updating
        # if mw is None, don't do anything
        if not mw: pass
        # otherwise, traverse trie again, but update mostSearched as necessary
        else:
            mw.searchCount += 1
            self._updateMostSearched(mw)
        
        return mw
    
    # helper function for search to find mw based on melody
    def _find(self, melody):
        '''
        Returns a musical work, and whether it is an exact match or not
        '''
        currNode = self.root
        
        # go through intervals (traverse trie)
        for ivl in melodyToIntervals(melody):
            # this node is a dead end
            if currNode.nextArr is None: return currNode.mostSearched, False
            # if not a dead end, keep going
            currNode = currNode.nextArr[ivlToIdx(ivl)]
        
        # if we didn't reach dead end, check if node is terminal
        if currNode.terminalValue: return currNode.terminalValue, True
        else: return currNode.mostSearched, False # no exact match

    #helper function for search to update mostSearched
    def _updateMostSearched(self, mw):
        currNode = self.root
        mS, sC = currNode.mostSearched, mw.searchCount
        
        # go through the intervals (traverse trie)
        for ivl in mw.melodyIntervals:
            # update mostSearched if necessary
            if mS is not None and sC > mS.searchCount:
                currNode.mostSearched = mw
            
            # go to next (no check as we know mw exists in trie)
            currNode = currNode.nextArr[ivlToIdx(ivl)]

## Compact Trie

- intervals member variable is an array, can have multiple intervals

In [38]:
class CompactNode:
    def __init__(self, ivls, prev, nextArr=None):
        '''
        ivls can be either an int (one interval) or an array (1+ intervals)
        '''
        if type(ivls) is int: self.intervals = [ivls]
        else: self.intervals = ivls
        
        self.prev = prev
        self.nextArr = nextArr
        
        # for if there is an item that terminates at this node
        # TODO: may need to make this an array
        self.terminalValue = None
        
        # to keep track of most searched item
        # also used for matching with searches that are only first half of the melody, etc.
        self.mostSearched = None
    
    def __str__(self):
        hasNext = 'a' if self.nextArr is not None else 'no'
        tV_msg = self.terminalValue.title if self.terminalValue is not None else 'None'
        mS = self.mostSearched
        
        if self.mostSearched: mS_msg = f'{mS.title} with {mS.searchCount} searches'
        else: mS_msg = 'None'
        
        out = f'Intervals {self.intervals}, with {hasNext} nextArr\n'
        out += f'Terminal value of {tV_msg}\n'
        out += f'Most searched is {mS_msg}'
        return out

In [39]:
class CompactTrie:
    def __init__(self):
        self.root = CompactNode(None, None)
        self.root.nextArr = createNewNextArr(self.root)
        self.numItems = 0
        
    def __str__(self):
        return f'Compact Trie with {self.numItems} items'
    
    def __len__(self):
        return self.numItems
    
    
    def insert(self, mw):
        print('inserting', mw.title)
        currNode = self.root
        mI = mw.melodyIntervals # shave off intervals as we process them
        
        # go through intervals (traverse trie)
        while len(mI) > 0:
            print('mI:', mI)
            # update most searched mw for this node, if None
            if not currNode.mostSearched: currNode.mostSearched = mw
            
            
            # figure out how much we match with current node
            mI_len = 0 if (mI is None) else len(mI)
            cN_len = 0 if (currNode.intervals is None) else len(currNode.intervals)
            firstDiff = firstNonmatching(mI, currNode.intervals)
            print(f'mI len = {mI_len}, cN len = {cN_len}')
            
            # - Case 1: arrays not equal somewhere in the middle
            #           = "split" the node
            if firstDiff != -1:
                print('Case 1')
                # create parent node
                pNode = CompactNode(mI[:firstDiff], currNode.prev)
                pNode.nextArr = createNewNextArr(prev=pNode)
                
                # update currNode's prev's nextArr to point to new parent node
                currNode.prev.nextArr[ivlToIdx(mI[0])] = pNode
                
                # update currNode's prev and intervals
                currNode.prev = pNode
                currNode.intervals = currNode.intervals[firstDiff:]
                
                # insert currNode into new parent node's nextArr
                currNodeIdx = ivlToIdx(currNode.intervals[0])
                pNode.nextArr[currNodeIdx] = currNode # link currNode as a next node
                
                pNode.mostSearched = currNode.mostSearched

                # *step down trie*, to corresponding child of parent node
                mI = mI[firstDiff:]
                currNode = pNode.nextArr[ivlToIdx(mI[0])]
            
            # - Case 2: equal, but more remaining ivls than in this node's ivl array
            elif mI_len > cN_len:
                # Case 2a: we have next array
                #          = *step down trie*
                if currNode.nextArr is not None:
                    print('Case 2a')
                    mI = mI[cN_len:]
                    currNode = currNode.nextArr[ivlToIdx(mI[0])]

                # Case 2b: no next array, currNode not a terminal node
                #          = extend node's ivls, DONE
                elif not currNode.terminalValue:
                    print('Case 2b')
                    currNode.intervals = mI
                    break

                # Case 2c: no next array, currNode is a terminal node
                #          = create next array, add child node into array, DONE
                else:
                    print('Case 2c')
                    currNode.nextArr = createNewNextArr(prev=currNode)
                    cNode = CompactNode(mI[cN_len:], currNode)
                    currNode.nextArr[ivlToIdx(mI[0])] = cNode
                    
                    currNode = cNode
                    break

            # - Case 3: equal, but less remaining ivls than in this nodes' ivl array
            #           = "split" the node, DONE
            elif mI_len < cN_len:
                print('Case 3')
                # create new parent node
                pNode = CompactNode(mI, currNode.prev)
                pNode.nextArr = createNewNextArr(prev=pNode)
                
                # update currNode's prev's nextArr to point to new parent node
                currNode.prev.nextArr[ivlToIdx(mI[0])] = pNode
                
                # update currNode's prev and intervals
                currNode.prev = pNode
                currNode.intervals = currNode.intervals[mI_len:]
                
                # insert currNode into new parent node's nextArr
                currNodeIdx = ivlToIdx(currNode.intervals[0])
                pNode.nextArr[currNodeIdx] = currNode # link currNode as a next node
                
                pNode.mostSearched = currNode.mostSearched
                
                # currNode is now pNode
                currNode = pNode
                break

            # - Case 4: identical. Currently not possible, as this would imply duplicates
            # TODO: change terminalValue into array?
            else: return
        
        
        # currNode now holds terminal node for this musical work
        currNode.terminalValue = mw
        self.numItems += 1 # increment number of items
    
    def search(self, melody):
        # try to find the musical work
        # TODO: for speed reasons, find out how to return the result first,
        #       and only then update mostSearched?
        mw, exact = self._find(melody)
        
        
        # Determine if any Nodes' mostSearched needs updating
        # if mw is None, don't do anything
        if not mw: pass
        # otherwise, traverse trie again, but update mostSearched as necessary
        else:
            mw.searchCount += 1
            self._updateMostSearched(mw)
        
        return mw
    
    # helper function for search to find mw based on melody
    def _find(self, melody):
        '''
        Returns a musical work, and whether it is an exact match or not
        '''
        currNode = self.root
        
        # go through intervals (traverse trie)
        for ivl in melodyToIntervals(melody):
            # this node is a dead end
            if not currNode.nextArr: return currNode.mostSearched, False
            # if not a dead end, keep going
            currNode = currNode.nextArr[ivlToIdx(ivl)]
        
        # if we didn't reach dead end, check if node is terminal
        if currNode.terminalValue: return currNode.terminalValue, True
        else: return currNode.mostSearched, False # no exact match

    #helper function for search to update mostSearched
    def _updateMostSearched(self, mw):
        currNode = self.root
        mS, sC = currNode.mostSearched, mw.searchCount
        
        # go through the intervals (traverse trie)
        for ivl in mw.melodyIntervals:
            # update mostSearched if necessary
            if mS is not None and sC > mS.searchCount:
                currNode.mostSearched = mw
            
            # go to next (no check as we know mw exists in trie)
            currNode = currNode.nextArr[ivlToIdx(ivl)]

# Testing

In [40]:
# START MALLOC
# https://medium.com/survata-engineering-blog/monitoring-memory-usage-of-a-running-python-program-49f027e3d1abmess
tracemalloc.start()

In [58]:
t = Trie()

In [59]:
# Define some pieces/songs
works = []

works.append(ClassicalPiece('Fur Elise', 'Ludwig van Beethoven', 1810, 'Classical Period', 'a minor',
                            'E5 D#5 E5 D#5 E5 B4 D5 C5 A4 / C4 E4 A4 B4 / E4 G#4 B4 C5',
                            None, None))

works.append(Song('Happy Birthday!', None, None, None, None, 'C4 C4 D4 C4 F4 E4'))
works.append(Song('Twinkle Twinkle Little Star', None, None, None, None, 'C4 C4 G4 G4 A4 A4 G4'))

works.append(Song('Avatar\'s Love', None, 2003, 'TV Show Music', 'C major', 'C5 B4 G4 E4'))
# works.append(Song('Jurrasic Park Theme', 'John Williams', 1993, 'Movie Score', 'C major',
#                   'C5 B4 C5 G4 F4 C5 B4 C5 G4 F4 C5 B4 C5 D5 D5 F5 F5 / E5 C5 D5 B4 G4 E5 C5 D5 / ' + 
#                   'G5 C5 F5 E5 E5 D5 D5'))
works.append(Song('Jurrasic Park Theme', 'John Williams', 1993, 'Movie Score', 'C major',
                  'C5 B4 C5 G4 F4 C5 B4 C5 G4 F4 C5 B4 C5 D5 D5 F5 F5'))
works.append(Song('Le Festin', 'Ratatouille', 2000, 'Movie Score', 'Bb major',
                  'Bb3 G4 F4 Eb4 G4 F4 Eb4 G4 F4 Eb4 Bb4'))
works.append(Song('Jazz Lick', None, None, 'Jazz', 'C major', 'C3 E2 F2 F#2 G2 A2 B2 C2'))

works.append(MusicalWork('custom', 'bruh', 2021, 'yerp', 'C major', 'C5 B4 C5'))

works.append(MusicalWork('random1', 'me', 2021, None, None, 'A4 B4 C5'))
works.append(MusicalWork('random2', 'me', 2021, None, None, 'C4 D4 E4'))
works.append(MusicalWork('random3', 'me', 2021, None, None, 'D4 D4 A4 A4 D5 A4 D4'))
works.append(MusicalWork('random4', 'me', 2021, None, None, 'Gb6 Bb6 Db7'))
works.append(MusicalWork('random5', 'me', 2021, None, None, 'F6 C6 A5 F5'))

# insert
for mw in works: t.insert(mw)

In [60]:
print(t.search('G3 G3 A3 G3'))

[+1] Happy Birthday! | by None (None, None)
Melody [None]: C4 C4 D4 C4 F4 E4


In [61]:
print(t.search('Bb5 G6 F6 Eb6'))

[+1] Le Festin | by Ratatouille (2000, Movie Score)
Melody [Bb major]: Bb3 G4 F4 Eb4 G4 F4 Eb4 G4 F4 Eb4 Bb4


In [48]:
print(t.search('E5 D#5 E5 D#5 E5 B4 D5 C5 A4'))

[+1] Fur Elise | by Ludwig van Beethoven (1810, Classical Period)
Melody [a minor]: E5 D#5 E5 D#5 E5 B4 D5 C5 A4 / C4 E4 A4 B4 / E4 G#4 B4 C5


In [214]:
print(t.search('G4 F#4 G4 D4 C4'))

[+1] Jurrasic Park Theme | by John Williams (1993, Movie Score)
Melody [C major]: C5 B4 C5 G4 F4 C5 B4 C5 G4 F4 C5 B4 C5 D5 D5 F5 F5


In [215]:
print(t.search('Gd3 G3'))

[+1] random1 | by me (2021, None)
Melody [None]: A4 B4 C5


In [216]:
# STOP MALLOC
current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage is {current / 10**3}kB; Peak was {peak / 10**3}kB")
tracemalloc.stop()

Current memory usage is 1591.625kB; Peak was 1648.448kB


In [208]:
# STOP MALLOC
current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage is {current / 10**3}kB; Peak was {peak / 10**3}kB")
tracemalloc.stop()

Current memory usage is 2041.372kB; Peak was 2098.595kB


# Compact Trie Testing

In [92]:
class MW_Test:
    def __init__(self, title, melodyIntervals):
        self.title = title
        self.melodyIntervals = melodyIntervals
        
        self.searchCount = 0
    
    def __str__(self):
        return f'{title}: {melodyIntervals}'

In [139]:
testNames = ['base', 'short substring', 'long substring',
             'firstDiff = 4, longer', 'firstDiff = 2, longer', 'firstDiff = 2, shorter',
             'different', 'different2']

ivlsList = [[1,2,3,4,5,6], [1], [1,2,3,4],
            [1,2,3,7,8,9,8,7], [1,2,7,7,7,6,6,5], [1,2,7],
            [0,5,9,8], [0,5,8,7]]

for i in range(len(testNames)):
    print(testNames[i], ':', ivlsList[i])

base : [1, 2, 3, 4, 5, 6]
short substring : [1]
long substring : [1, 2, 3, 4]
firstDiff = 4, longer : [1, 2, 3, 7, 8, 9, 8, 7]
firstDiff = 2, longer : [1, 2, 7, 7, 7, 6, 6, 5]
firstDiff = 2, shorter : [1, 2, 7]
different : [0, 5, 9, 8]
different2 : [0, 5, 8, 7]


In [140]:
ct = CompactTrie()

In [142]:
testOrder = [0,1,2]

for i in testOrder:
    testName = testNames[i]
    ivls = ivlsList[i]
    ct.insert(MW_Test(testName, ivls))

inserting base
mI: [1, 2, 3, 4, 5, 6]
mI len = 6, cN len = 0
Case 2a
mI: [1, 2, 3, 4, 5, 6]
mI len = 6, cN len = 1
Case 2b
inserting short substring
mI: [1]
mI len = 1, cN len = 0
Case 2a
mI: [1]
mI len = 1, cN len = 6
Case 3
inserting long substring
mI: [1, 2, 3, 4]
mI len = 4, cN len = 0
Case 2a
mI: [1, 2, 3, 4]
mI len = 4, cN len = 1
Case 2a
mI: [2, 3, 4]
mI len = 3, cN len = 5
Case 3


In [115]:
print(ct.root.nextArr[ivlToIdx(1)].nextArr[ivlToIdx(2)].nextArr[ivlToIdx(5)])

Intervals [5, 6], with no nextArr
Terminal value of base
Most searched is base with 0 searches
